# Get Huggingface pretrained model

In [1]:
!pip install transformers --upgrade --quiet

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import transformers
import torch
from transformers import pipeline

## Get pretrained model and save it

In [3]:
pretrained_classifier = pipeline("sentiment-analysis")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
pretrained_classifier([
    "I really like this place!",
    "The food was bad!"
])

[{'label': 'POSITIVE', 'score': 0.9998599290847778},
 {'label': 'NEGATIVE', 'score': 0.9997907280921936}]

In [5]:
pretrained_classifier.save_pretrained("./model/")

## Check if we can successfully load the saved model

In [6]:
local_classifier = pipeline('sentiment-analysis', model="./model/", tokenizer="./model/")

In [7]:
local_classifier([
    "I really like this place!",
    "The food was bad!"
])

[{'label': 'POSITIVE', 'score': 0.9998599290847778},
 {'label': 'NEGATIVE', 'score': 0.9997907280921936}]

## Build container for Lambda

In [8]:
!python --version
print(f"Torch version {torch.__version__}")
print(f"Transformers version {transformers.__version__}")

Python 3.6.13
Torch version 1.7.1
Transformers version 4.9.2


In [9]:
!pip freeze > requirements.txt

In [10]:
%%writefile requirements.txt
--find-links  https://download.pytorch.org/whl/torch_stable.html 
 
torch==1.7.1+cpu
transformers==4.9.2

Overwriting requirements.txt


### Get a base image for huggingface and pytorch

In [11]:
# from sagemaker.huggingface import HuggingFaceModel
# from sagemaker import get_execution_role
# import boto3

In [12]:
# hub = {
# #   'HF_MODEL_ID':'distilbert-base-uncased-distilled-squad',
#   'HF_TASK':'"sentiment-analysis"'
# }
# # create Hugging Face Model Class
# huggingface_model = HuggingFaceModel(
#     transformers_version='4.9',
#     pytorch_version='1.7',
#     py_version='py36',
#     env=hub,
#     role=get_execution_role(),
# )

In [13]:
# boto3_session = boto3.session.Session()
# current_region = boto3_session.region_name
# base_image = huggingface_model.serving_image_uri(current_region, instance_type="ml.c5.medium")
# base_image

### Write dockerfile

https://docs.aws.amazon.com/lambda/latest/dg/images-create.html

In [14]:
%%writefile Dockerfile
FROM public.ecr.aws/lambda/python:3.8

COPY requirements.txt ./requirements.txt
RUN pip install -r requirements.txt 

COPY ./app/app.py   ./
CMD ["app.handler"]

Overwriting Dockerfile


In [15]:
# !sudo chown -R $USER .

In [16]:
# !docker build -f Dockerfile .

## Build dockerfile in codebuild and push to ECR

In [17]:
!pip install sagemaker-studio-image-build --quiet

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


make sure this role has a trust relationship with codebuild

https://github.com/aws-samples/sagemaker-studio-image-build-cli

In [18]:
!sm-docker build . --repository huggingface-on-lambda:1.0

...[Container] 2021/08/20 07:23:11 Waiting for agent ping

[Container] 2021/08/20 07:23:14 Waiting for DOWNLOAD_SOURCE
[Container] 2021/08/20 07:23:17 Phase is DOWNLOAD_SOURCE
[Container] 2021/08/20 07:23:17 CODEBUILD_SRC_DIR=/codebuild/output/src118816575/src
[Container] 2021/08/20 07:23:17 YAML location is /codebuild/output/src118816575/src/buildspec.yml
[Container] 2021/08/20 07:23:17 Processing environment variables
[Container] 2021/08/20 07:23:17 No runtime version selected in buildspec.
[Container] 2021/08/20 07:23:17 Moving to directory /codebuild/output/src118816575/src
[Container] 2021/08/20 07:23:17 Registering with agent
[Container] 2021/08/20 07:23:17 Phases found in YAML: 3
[Container] 2021/08/20 07:23:17  PRE_BUILD: 9 commands
[Container] 2021/08/20 07:23:17  BUILD: 4 commands
[Container] 2021/08/20 07:23:17  POST_BUILD: 3 commands
[Container] 2021/08/20 07:23:17 Phase complete: DOWNLOAD_SOURCE State: SUCCEEDED
[Container] 2021/08/20 07:23:17 Phase context status code:  M

You should consider upgrading via the '/var/lang/bin/python3.8 -m pip install --upgrade pip' command.
Removing intermediate container 2edb23706805
 ---> db02a276579f
Step 4/5 : COPY ./app/app.py   ./
 ---> 6198e98a12e7
Step 5/5 : CMD ["app.handler"]
 ---> Running in 4114fc7c41c1
Removing intermediate container 4114fc7c41c1
 ---> 1c7e7c6064c2
Successfully built 1c7e7c6064c2
Successfully tagged huggingface-on-lambda:1.0

[Container] 2021/08/20 07:24:34 Running command docker tag $IMAGE_REPO_NAME:$IMAGE_TAG $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG

[Container] 2021/08/20 07:24:34 Phase complete: BUILD State: SUCCEEDED
[Container] 2021/08/20 07:24:34 Phase context status code:  Message:
[Container] 2021/08/20 07:24:34 Entering phase POST_BUILD
[Container] 2021/08/20 07:24:34 Running command echo Build completed on `date`
Build completed on Fri Aug 20 07:24:34 UTC 2021

[Container] 2021/08/20 07:24:34 Running command echo Pushing the Docker image

## Deploy Container within a Lambda Function

create a serverless.yml file that points to the docker container and expects a POST request on the path "prediction/"

In [11]:
%%writefile serverless.yml
service: huggingface-on-lambda

provider:
  name: aws
  region: eu-west-1 
  memorySize: 1024 
  timeout: 30 

functions:
  huggingface:
    # paste the <image-uri>@<image-digest>
    image: 077590795309.dkr.ecr.eu-west-1.amazonaws.com/huggingface-on-lambda@sha256:8759164fb525c94bf5c74cfc961053182ec0359513e244d654dbbbbf09c0a9d1
    events:
      - http:
          path: prediction # http path
          method: post # http method



Overwriting serverless.yml


open an AWS cloudshell and install the serverless framework

    npm install serverless

create the serverless.yml file in the cloudshell
    
    cat > serverless.yml

paste the contents and press ctrl + c to close and save the file.
To deploy the stack run

    serverless deploy